In [13]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import os
import sys
import time
from multiprocessing import Process, Pool
import warnings
warnings.simplefilter('ignore')

# Add module path to system path
module_paths = ['..', 
                '../..',
               ]
for module_path in module_paths:
    if os.path.abspath(os.path.join(module_path)) not in sys.path:
        sys.path.insert(0, module_path)
    
from epn_mining.preparation import epn
from epn_mining.utils.io import load, save
    


# Set session variables
verbose = True
state_store=True


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec

from tqdm import tqdm

import cfod
from cfod import catalog
from cfod.routines import waterfaller

import h5py
import scipy
import wget

In [15]:
data_catalog = catalog.as_dataframe()
#list(data_catalog['tns_name'])

In [16]:
from Graph_getter import Get_me_FRB_data

In [5]:
plot_time, ts_full_list, model_ts_full_list, times_shorter, cal_ts_shorter, snr = Get_me_FRB_data(10)
plot_time

file not in folder, downloading
100% [........................................................................] 35527920 / 35527920Peak: 14 at time sample, Width = 4.915199970128015 ms, SNR = 279.1211697946155


/Users/vohl/Documents/code/graph_profiles/notebooks/Graph_getter.py:101: RuntimeWarning: Degrees of freedom <= 0 for slice.
  channel_variance = np.nanvar(wfall, axis=1)


array([-12.28799993, -11.30495993, -10.32191994,  -9.33887994,
        -8.35583995,  -7.37279996,  -6.38975996,  -5.40671997,
        -4.42367997,  -3.44063998,  -2.45759999,  -1.47455999,
        -0.49152   ,   0.49152   ,   1.47455999,   2.45759999,
         3.44063998,   4.42367997,   5.40671997,   6.38975996,
         7.37279996,   8.35583995,   9.33887994,  10.32191994,
        11.30495993,  12.28799993,  13.27103992,  14.25407991,
        15.23711991,  16.2201599 ,  17.2031999 ,  18.18623989,
        19.16927988,  20.15231988,  21.13535987,  22.11839987,
        23.10143986,  24.08447985,  25.06751985])

In [18]:
import wget
wget.download?

Signature: wget.download(url, out=None, bar=<function bar_adaptive at 0x7f8c846b4f70>)
Docstring:
High level function, which downloads URL into tmp file in current
directory and then renames it to filename autodetected from either URL
or HTTP headers.

:param bar: function to track download progress (visualize etc.)
:param out: output filename or directory
:return:    filename where URL is downloaded to
File:      ~/.local/lib/python3.8/site-packages/wget.py
Type:      function


In [46]:
#The bins are currently set as:

LIMS = (
   [0,  200], # in MHz
   [200,400],
   [400,700],
   [700,1000],
   [1000,1500],
   [1500,2000],
   [2000,1199169832000000]
)
LIMS_DICT = {i:v for i, v in enumerate(LIMS)}

def get_freq_range(freq):
    for freq_bin in LIMS_DICT.keys():
        l, h = LIMS_DICT[freq_bin]
        if (freq >= l) and (freq < h):
            return freq_bin
    return None

In [10]:
df_repeaters = data_catalog.loc[data_catalog.repeater_name != '-9999']
df_repeaters.index

Int64Index([ 12,  22,  25,  30,  33,  34,  35,  38,  61,  65,  66,  75,  76,
             77,  78,  79,  81,  82,  87,  88, 100, 103, 104, 105, 106, 108,
            109, 120, 121, 157, 162, 163, 164, 165, 171, 172, 181, 184, 185,
            211, 239, 240, 245, 266, 295, 296, 297, 298, 299, 302, 310, 311,
            313, 314, 319, 335, 354, 355, 360, 372, 419, 441, 449, 450, 480,
            484, 489, 490, 497, 498, 499, 500, 501, 502, 503, 508, 522, 524,
            526, 527, 528, 533, 534, 535, 536, 537, 547, 548, 566, 577, 578,
            579, 582, 592],
           dtype='int64')

In [43]:
get_freq_range(600)

600 0 200
600 200 400
600 400 700


2

In [47]:
from epn_mining.preparation.pulsar import Pulsar, Population, Observation

population = Population()

df_repeaters = data_catalog.loc[data_catalog.repeater_name == '-9999']


for index in tqdm(df_repeaters.index):
    pulsar = Pulsar(jname = df_repeaters['tns_name'][index], index = index)
    #print(pulsar.jname)
    
    _, _, model_ts_full_list, _, _, snr = Get_me_FRB_data(index)
    
    if model_ts_full_list is not None:
        obs=Observation(frequency=600, frequency_range=get_freq_range(600))

        obs.model = (model_ts_full_list - np.min(model_ts_full_list)) / (np.max(model_ts_full_list) - np.min(model_ts_full_list))
        obs.profile = model_ts_full_list
        obs.snr = snr

        pulsar.add_observation(obs)

        population.add_pulsar(pulsar)
    
    # print(index)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 506/506 [01:24<00:00,  5.98it/s]


In [48]:
save('population', population, state_prefix='FRB_one_offs')

In [ ]:
data_cal

In [12]:
# from epn_mining.topology import topology

# # If you set the Observation frequency to 600 (MHz, central freq for CHIME), it will set the frequency id = 2
# freq_ids_to_include = [2]

# distances, \
# population_graph_indices, \
# graph_population_indices = topology.complete_undirected_weighted_graph(population,
#                                                                        metric=metric,
#                                                                        stokes_to_include=stokes_to_include,
#                                                                        weights=weights,
#                                                                        freq_ids_to_include=freq_ids_to_include,
#                                                                        min_snr=min_snr,
#                                                                        penalty=penalty,
#                                                                        step_pattern=step_pattern,
#                                                                        window_type=window_type,
#                                                                        window_args=window_args,
#                                                                        open_begin=open_begin,
#                                                                        open_end=open_end,
#                                                                        cropped=cropped,
#                                                                        verbose=verbose)


In [11]:
population = load('population', state_prefix='FRB')


In [12]:
population